機械学習を用いたLinear regressionの基本的な考え方とhaskellでの実装を示す。（Floatでやるべきかtensorでやるべきか。。）
haskellでのコードは、すべて今回の問題設定に則ったvalidなデータのみが入力されることを想定している。

その後hasktorchでの実装をみていく。

## 1. What is Linear Regression?

Let's consider the revenue estimation for a supermarket in front of a station.

The table below shows revenue for 10 months and the items that affect it[<sup>1</sup>](#id_01):
- The number of passengers (because there is a supermarket in front of the station) 
- The amount of goods.

| month | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Revenue | 123 | 290 | 230 | 261 | 140 | 173 | 133 | 179 | 210 | 181 |
| Passengers | 9.3K | 23K | 25K | 26K | 11.9K | 18.3K | 15.1K | 19.2K | 26.3K | 18.5K |
| Goods | 150 | 311 | 182 | 245 | 152 | 162 | 99 | 184 | 115 | 105 |

<!-- In order to predict it with machine learning, we need past revenue and past data items that can affect it. For example, in this case, the passenger of the station and Amount of goods is considered  -->

<!-- With this data, we can see the relationship between revenue, passengers, and products for 10 months. In other words, we can obtain expression for them. -->

In order to estimate revenue based on this data, we need to express those relationships using approximate formula. <br>
In this case, revenue seems to be proportional to passengers and goods, so we'll analyze assuming that there is a linear relationship between them, which is called **Linear Regression**.[<sup>2</sup>](#id_02)

To define an approximate expression, suppose that a set of month is $M=\{1,\dots,10\}$, the number of passenger, the amount of goods and revenue are $(x_i)_{i\in M},(y_i)_{i\in M},(z_i)_{i\in M}$. i.e. $(x_i)_{i\in M}=(9.3,23,\dots,18.5)$, $(y_i)_{i\in M}=(150,311,\dots,105)$, $(z_i)_{i\in M}=(123,290,\dots,181)$ <br>
In linear regression, it is assumed that there is a [linear mapping](https://www.ucl.ac.uk/~ucahmto/0005_2021/Ch4.S14.html) expressed by <br>
$z = a x + b y + c$<br>
and each $z_i$ will be approximated by the value $ax_i+by_i+c$.

Therefore, for estimating the revenue from passenger($x_j$) and good($y_j$) for a given month($j$), below linear function is used.
$$
f = ax_j+by_j+c
$$

Now, let's define the data and the linear function in haskell.<br>
This lecture use [Tensor type in hasktorch](https://hasktorch.github.io/tutorial/02-tensors.html), which is multidimensional array with a fixed shape and element type.

In [72]:
-- data
import Torch.Tensor (Tensor, asTensor)

z :: Tensor
z = asTensor [[123, 290, 230, 261, 140, 173, 133, 179, 210, 181]]

x :: Tensor
x = asTensor [9300, 23000, 25000, 26000, 11900, 18300, 15100, 19200, 26300, 18500]
y :: Tensor
y = asTensor [150, 311, 182, 245, 152, 162, 99, 184, 115, 105]

Since they are handled as Tensor type, matrix operations are performed in the implementation for linear function.<br>
Suppose that $s = [a, b]$ (1 × 2 matrix) and $v = [x, y]$　(2 × 10 matrix),
$$
f = s\cdot v + c
% \begin{aligned}
% \mathcal{L} &= \cfrac{1}{n(M)}\sum_i(z_i - \vec{p}\cdot\vec{v_i})^2 \\
%             &= \cfrac{1}{n(M)}\sum_i(\vec{p}\cdot\vec{v_i} - z_i)^2 
% \end{aligned}
$$
 

In [77]:
import Torch.Functional (matmul, add, transpose2D)
-- | Predict revenue from the passengers and goods,
linear :: 
    (Tensor, Tensor) -> -- ^ parameters ([a, b]: 1 × 2, c: scalar)
    Tensor ->           -- ^ data (passengers, goods): 2 × 10
    Tensor              -- ^ estimated value: 1 × 10
linear (slope, intercept) input = (slope `matmul` input) `add` intercept

Example when $a = 2$, $b = 3$, and $c = 4$:

In [101]:
import Torch.Functional (Dim(..) , stack)
sampleSlope = asTensor [[0.006, 0.4]]
sampleIntercept = asTensor [0.1]

sampleEstimation = linear (sampleSlope, sampleIntercept) (stack (Dim 0) [x, y])
sampleEtimation

Tensor Double [1,10] [[ 115.9000   ,  262.5000   ,  222.9000   ,  254.1000   ,  132.3000   ,  174.7000   ,  130.3000   ,  188.9000   ,  203.9000   ,  153.1000   ]]

Here, we can now get an estimate of revenue given a passenger and goods for a given month. However, the parameters a, b, and c still remain unknown.

## 2. How can we get better parameters?

To estimate revenue more accurately, we need to obtain the parameters a, b, and c that best approximate the data. In other words, the parameters minimize the discrepancy between the actual data($z$) and the output of the expression($ax_i+by_i+c$). <br>
There are multiple solution methods, but here we will analyze using machine learning.

TODO:  add graph like http://www2.toyo.ac.jp/~mihira/keizaitoukei2014/ols1.pdf

The discrepancy of each data data is $|\delta i|$, and we consider the sum of squares divided by the number of data (i.e. the number of month $= n(M)$) as the function to be minimized.<br>
The function of sum of squares divided by the number of data is called **Mean Squared Error** and is often used in regression analysis. The function to be minimized is called **Loss Function($=\mathcal{L}$)**.
$$
\begin{aligned}
\mathcal{L} &= \cfrac{1}{n(M)}\sum_i\delta_i^2 \\
            &= \cfrac{1}{n(M)}\sum_i(z_i - ax_i - by_i - c)^2 \\
\end{aligned}
$$

This function can be defined in haskell as below:

In [102]:
import Torch (size)
import Prelude hiding (div)
import Torch.Functional (pow, div)

-- TODO: 微分するためには、a,b cそれぞれパラメータとして持っとく必要がある？

loss ::
    Tensor -> -- ^ actual values: 1 × 10
    Tensor -> -- ^ estimated values: 1 × 10
    Tensor    -- ^ loss: scalar
loss z z' = pow 2 (z - z') `div` asTensor (size 1 z)

The loss between z and `sampleEstimation` is:

In [105]:
loss z sampleEstimation

Tensor Double [1,10] [[ 5.0410   ,  75.6250   ,  5.0410   ,  4.7610   ,  5.9290   ,  0.2890   ,  0.7290   ,  9.8010   ,  3.7210   ,  77.8410   ]]

In order to minimize $\mathcal{L}$, we partially differentiate $\mathcal{L}$ by the parameters and find the value that points to the lowest point (minimum value). This value is called **Gradient** ($=\nabla \mathcal{L} $).
$$
\nabla \mathcal{L} = \left (\cfrac{\partial \mathcal{L}}{\partial a}, \cfrac{\partial \mathcal{L}}{\partial b}, \cfrac{\partial \mathcal{L}}{\partial c} \right)
$$

The function to calculate the gradient is as below:

In [121]:
-- | Calculate numerical gradient when the argument of f is x.
-- it's partially differentiated for each elament of x.
numericalGradient　::
    (Tensor -> Tensor) -> -- ^ differentiated function
    Tensor             -> -- ^ argument of the function
    Tensor
numericalGradient f x = 
    f (x `add` h) + f (x `add` h) / 2
    where
        h = 1e-4

In [119]:
import Torch.Tensor ((!), Slice(..))
z ! Slice (0,2)

Tensor Double [1,10] [[ 123.0000   ,  290.0000   ,  230.0000   ,  261.0000   ,  140.0000   ,  173.0000   ,  133.0000   ,  179.0000   ,  210.0000   ,  181.0000   ]]

You might think that it is a good idea to solve actual value to solve the equations. This is fine for linear regression, however for many real-world problems, the parameter space is so vast that finding the minimum point is difficult.<br>
Therefore, machine learning uses optimization algorithm　that efficiently search for the minimum value.[<sup>3</sup>](#id_03) <br>
Here we use the simplest one: **Gradient Descent**. This algorithm varies the parameters along the gradient by a certain distance. Then, repeat the process of finding the gradient and changing the parameters at the destination.
That is, update the parameters as below:
$$
(a, b, c) = \left (a - \eta\cfrac{\partial \mathcal{L}}{\partial a}, b - \eta\cfrac{\partial \mathcal{L}}{\partial b}, c - \eta\cfrac{\partial \mathcal{L}}{\partial c} \right)
$$
$\eta$ represent the update amount which is called **Learning Rate**. This determines how far the parameter moves in one step. [<sup>4</sup>](#id_04)


Then, let's write a code to estimate the revenue using functions we've defined.

In [ ]:
gradientDiscent

1. <span id="id_01"> https://www.statweb.jp/method/kaiki-bunseki/senkeikaiki-jyu-case </span>
2. <span id="id_02">The method differs depending on the relationship assumed. (For more variation, see non-regression analysis](https://en.wikipedia.org/wiki/Nonlinear_regression))</span>
3. <span id="id_03">Of course, using an optimization algorithm does not always guarantee that we will actually get the minimum value. When a function is complex, it may fall into a local minimum. Various [optimization algorithm](https://d2l.ai/chapter_optimization/) have been developed to solve this problem.</span>
4. <span id="id_04">This is one of the manually defined hyperparameters, unlike parameters (a, b, and c).</span>

参照
http://www2.toyo.ac.jp/~mihira/keizaitoukei2014/ols1.pdf
http://fs1.law.keio.ac.jp/~aso/ecnm/pp/reg2.pdf